# Relatório Final

## 1 Heurística de Alinhamento Local de Smith-Waterman

O Algoritmo de Alinhamento Local de Smith-Waterman requer a utilização de uma matriz para sua implementação.
 Para duas sequências de tamanhos $N$ e $M$, a matriz tem formato $(N + 1)x(M + 1)$.

A matriz é definida com a primeira linha e a primeira coluna composta apenas de zeros. A partir disso, um
 elemento $S_{i,j}$ é dado por:

$$ 
  S_{i,j} = max\begin{Bmatrix}
  S_{i-1, j-1} + 2, & a_i = b_j \\ 
  S_{i-1, j-1} - 1,  & a_i \neq b_j\\ 
  S_{i-1, j} - 1 &  b_j = -\\
  S_{i, j-1} - 1 &  a_i = -\\ 
  0 & 
  \end{Bmatrix}
$$

Sendo $a_i$ e $b_j$ o i-ésimo e o j-ésimo caractere, respectivamente das duas sequências de entrada.

### 1.1 Implementação

A implementação desse algoritmo pode ser dividida em 3 etapas:

- Ler arquivos de entrada;
- Inicializar a matriz zerada;
- Iterar sobre todos os elementos da matriz atualizando seus valores com base nos
  _scores_ que estão dispostos imediatamente anteriores (considerando linha e coluna)
  de acordo com regra do item 1. Em cada etapa da iteração, verficamos se o novo valor de score
  é maior que o máximo anteriormente conhecido. Se for, atualizamos o máximo conhecido;

Em C++:

```cpp
int algoritmo_smith_waterman() {
  std::vector<std::vector<int>> matriz;
  inicializa_matriz(&matriz, N, M, 0); // inicializa as N linhas e M colunas com zeros
  int max_score = -1;
  // aplica algoritmo; atualiza score
  return max_score;
}

int calcula_score(std::string A, std::string B, int N, int M) {
  int max_score = algoritmo_smith_waterman(A, B, N, M);
  return max_score;
}

int main() {
  int N, M;

  std::cin >> N;
  std::cin >> M;

  std::string A;
  std::string B;

  std::cin >> A;
  std::cin >> B;  

  int score = calcula_score(A, B, N, M);

  std::cout << "Max score: " << score << std::endl;

  return 0;
}
```

## 2 Busca Local (Aleatorizada)

A estratégia de Busca Local Aleatorizada é particularmente útil para problemas que não
 necessitam da solução ótima ou que a solução ótima não é necessariamente. Um exemplo disso
 seria um par de sequências de DNA bem grandes com quase nenhuma subsequência em comum. Acertar
 a melhor fração comum entre as duas poderia levar bastante tempo se isso fosse feito de modo
 sequencial. No entanto, uma comprar um número razoável de subsequências aleatórias das duas
 poderia rapidamente atigir esse valor máximo.

### 2.1 Implementação

A implementação poderia ser feita da seguinta forma:

- Ler arquivo de entrada;
- Gerar várias subsequências aleatórias a partir das sequências de entrada;
  - Calcular o score com o algoritmo de Smith-Waterman;
  - verificar se esse valor é maior que o máximo já conhecido; 

Em C++:

```cpp

std::string random_subseq(std::string seq);

int main() {
  int N, M;
  std::cin >> N >> M;
  std::string A, B;
  std::cin >> A >> B;


  int K = 30; // número arbitrário testes aleatorizados a serem feitos

  int max_score = -1;
  for (int i = 0; i < K; i++) {
    std::string ssA = random_subseq(A);
    std::string ssB = random_subseq(B);
    int local_score = calcula_score(ssA, ssB, ssA.size(), ssB.size());
    if (local_score > max_score) max_score = local_score;
  }

  std::cout << "Max score: " << max_score << std::endl;
  return 0;
}
```

## 3 Busca Exaustiva

Esse método segue um algoritmo simples.

- Ler arquivo de entrada com sequências A e B;
- Gerar todas as subsequências de A e de B;
- Utilizar algum método para calcular score de cada uma das subseq. de A com cada uma de B;
  - O método pode ser uma simples comparação de caracteres ou pode ser a aplicação de _Smith-Waterman_;
  - Em cada uma das comparações, verificar e atualizar o maior score registrado, se necessário;

Em C++:

```cpp
std::string subseq(std::string seq);

int main() {
  int N, M;
  std::cin >> N >> M;
  std::string A, B;
  std::cin >> A >> B;

  int total_A = N * (N + 1) / 2; // total de subsequências de A
  int total_B = M * (M + 1) / 2; // total de subsequências de B

  std::vector<std::string> SA, SB; // vetores de subsequências de A e de B

  for (int start = 0; start < N; i++) {
    for (int length = 1; length <= N - start; length ++) {
      SA.push_back(subseq(A, start, length));
    }
  }
  for (int start = 0; start < M; i++) {
    for (int length = 1; length <= M - start; length ++) {
      SB.push_back(subseq(B, start, length));
    }
  }

  int max_score = -1;
  for (std::string ssA : SA) {
    for (std::string ssB : SB) {
      int local_score = calcula_score(ssA, ssB, ssA.size(), ssB.size());
      if (local_score > max_score) max_score = local_score;
    }
  }

  std::cout << "Max score: " << max_score << std::endl;
  
  return 0;
}
```

## 4 Paralelização OpenMP

Este método, utiliza a paralelização (com OpenMP) de algum dos métodos anteriores. 
 Sem perda de generalidade, foi escolhido o método de Busca Exaustiva. Desse modo,
 podemos alterar alguns trechos (como __loops__) do código para que rodem em múltiplos
 __cores__ simultaneamente.

Em C++:

```cpp

//  gerar subsequencias
#pragma omp parallel
{  
  #pragma omp master
  {
    #pragma omp task shared(SA)
    for (int start = 0; start < N; i++) {
      for (int length = 1; length <= N - start; length ++) {
        SA.push_back(subseq(A, start, length));
      }
    }
    #pragma omp task shared(SB)
    for (int start = 0; start < M; i++) {
      for (int length = 1; length <= M - start; length ++) {
        SB.push_back(subseq(B, start, length));
      }
    }
  }
}

// calcular score
#pragma omp parallel shared(max_score)
{
  #pragma omp for reduction(max : max_score)
  for (int index = 0; index < SA.size() * SB.size(); index++) {
    // truque para iterar nas duas listas utilizando um único iterador
    int indexA = (int) index / SB.size();
    int indexB = (int) index % SB.size();

    std::string ssA = SA.at(indexA);
    std::string ssB = SB.at(indexB);

    int local_score = calcula_score(ssA, ssB, ssA.size(), ssB.size());

    if (local_score > max_score) {
      #pragma omp critical
      max_score = local_score;
    }
  }
}

```

## 5 Busca Exaustiva Paralelizada com GPU

A estratégia de busca exaustiva com GPU é análoga à anterior, porém com algumas diferenças
principalmente na função que calcula score. As principais diferenças são:

- Não é mais necessário gerar a matriz completa para o cálculo de score de Smith-Waterman. 
  Como o cálculo depende apenas da linha atual e da anterior, ele pode ser dividido nos dois passos a seguir:
  $$ 
  S_{temp}(i,j) = max\begin{Bmatrix}
  S_{i-1, j-1} + 2, & a_i = b_j \\ 
  S_{i-1, j-1} - 1,  & a_i \neq b_j\\ 
  S_{i, j-1} - 1 &  b_j = -\\
  0 & 
  \end{Bmatrix}
  $$
  $$ 
    S_{i,j} = max\begin{Bmatrix}
    S_{temp}(i,j) & \\ 
    S_{temp}(i-1, j) - 1 &  \\
    0 & 
    \end{Bmatrix}
  $$
- O cálculo é feito com a utilização conjunta de operadores unários e binários da biblioteca
  `thrust` da NVIDIA, dependendo apenas da linha anterior da matriz;
- Operadores customizados (`functors`) podem ser utilizados no cálculo, fazendo melhor uso das
  vantagens de se trabalhar com __many core__. Mais detalhes podem ser encontrados no Relatório GPU.